# Hubbard Model

I'm planning on running VQE and Trotter simulation on a Hubbard Hamiltonian to better understand its structure. 

The Hubbard model is a simplification of the interactions of electrons in a solid. It can be used to explain how these interactions yield insulating, magnetic, and superconducting effects (though I don't understand any of this yet). 

**Update 1**: I started by defining my own classes/functions for creation/annihilation operators, the Jordan-Wigner transform, the square lattice for the Hubbard Hamiltonian, the Hubbard Hamiltonian's matrix representation, and the Variational Hamiltonian Ansatz (VHA). As expected, my code and computer's RAM wasn't enough to work with anything more than a 2x2 square lattice (which has 8 qubits, so exists 256-dimensional Hilbert space). What did surprise me was how much more efficient OpenFermion was, because when I tried it when SSHing onto a Google Cloud computer, I was able to quickly solve the ground state of up to a 2x10 square lattice Hubbard Hamiltonian. Most of the functions/classes I defined are still available in the tools/ folder if you want to take a look. 

In [1]:
import numpy as np
import scipy.linalg
import scipy.optimize 

from tools.utils import *
tol = 0.005 # Tolerance for elementwise equality of matrices

### Defining creation and annihilation operators 

Since we're describing electrons, the Hubbard Hamiltonian's creation and annihilation operators have the normal fermionic anticommutation relations, namely that for a fermion on site $j,k$ with spin $\sigma, \pi$, 

$$ \large \{ c_{j \sigma}, c^\dagger_{k \pi } \} = \delta_{jk} \delta_{\sigma \pi} \qquad \{ c^\dagger_{j\sigma}, c^\dagger_{k \pi} \}= \{ c_{j\sigma} c_{k\pi} \} = 0  $$

### Jordan-Wigner Transformation

We let $\mid 0 \rangle$ represent no electron in a spin-orbital, and we let $\mid 1 \rangle$ represent an electron existing in a spin-orbital. This allows us to define the creation operator as $(X-iY)/2$ and the annihilation operator as $(X+iY)/2$. The main problem with this is that the creation and annihilation operators don't anti-commute. To fix this, we prepend the operator with Pauli-$Z$ tensors. Because the Pauli operators anti-commute, this preserves our desired anti-commutation relation. 

## Defining the Hubbard Hamiltonian

Okay, now that we have those preliminaries out of the way, what is the Hubbard Hamiltonian? What's the motivation behind it? How is it useful? 

Answer these. 

The Hubbard Hamiltonian: 
$$ \large H = -t \sum_{<j, k>, \sigma} \Big( c^\dagger_{j\sigma} c_{k\sigma} + c^\dagger_{k \sigma} c_{j \sigma} \Big) + U \sum_j n_{j \uparrow} n_{j \downarrow} - \mu \sum_j \Big(n_{j\uparrow} + n_{j\downarrow} \Big) $$

The first term is kinetic energy, a fermion moving from one site to another. The symbol $<j, k>$ implies iterating over sites that are adjacent. 

The second term is interaction energy, additional energy for a doubly-occupied site. 

The third term is chemical potential, which controls the filling. **Hm, a lot of places don't have this third term (Wikipedia even). Why?**

### 2D Hubbard Hamiltonian on a Square Lattice 

Here is a model of a 2D Hubbard Hamiltontian on a square lattice from [1811.04476](https://arxiv.org/pdf/1811.04476.pdf). 

![](images/2d_hubbard.png)

We'll use OpenFermion's `HubbardSquareLattice` class to define our lattice. OpenFermion has a simpler function `fermi_hubbard()` to create a `FermionOperator` to describe our system, but this doesn't give us access to the specific hopping terms in the Hamiltonian (the terms in the first summation above), which we'll need for the Variational Hamiltonian Ansatz. 

The `HubbardSquareLattice` class has a useful method: `site_pair_iter(edge_type)`. We'll use the 'horizontal_neighbor' and 'vertical_neighbor' edge types. But we also need to differentiate between "even" and "odd" horizontal and vertical neighbors: even horizontal neighbors will be horizontal neighbors whose leftmost site has even index, and likewise for horizontal odd, vertical even, and vertical odd neighbors. To get this added specificity, we'll need to examine each item in the iterable generated by `site_pair_iter()` or call the `to_site_index(site)` method.   

In [2]:
from openfermion.utils import HubbardSquareLattice
HubbardSquareLattice?

In [3]:
# HubbardSquareLattice parameters
x_n = 2 
y_n = 2 
n_dofs = 1 # 1 degree of freedom for spin
periodic = 1 # Not sure what this is, periodic boundary conditions?
spinless = 0 # Has spin

lattice = HubbardSquareLattice(x_n, y_n, n_dofs=n_dofs, periodic=periodic, spinless=spinless)

Now, we'll create a `FermiHubbardModel` instance by passing it our `HubbardSquareLattice` instance defined above. 

To get the `FermionOperator` instance, we need to call `FermiHubbardModel.hamiltonian()`. The [documentation](https://openfermion.readthedocs.io/en/latest/openfermion.html#openfermion.hamiltonians.FermiHubbardModel) isn't that great here, but the [source code](https://github.com/quantumlib/OpenFermion/blob/master/src/openfermion/hamiltonians/_general_hubbard.py) indicates we do indeed get a `FermionOperator` instance which we'll need for calculating ground state, etc. 

We can't just pass an integer for $t$, $U$, or $\mu$ in this class. Instead, we have to specify the *specific* coefficient for each pair and edge type ($t_{ij}^{\textrm{horizontal neighbor}}$). This will be useful later on, I think, because [1506.05135](https://arxiv.org/abs/1506.05135) says we'll need the indices for different values during adiabatic evolution. 

In [4]:
from openfermion.hamiltonians import FermiHubbardModel
FermiHubbardModel??

In [5]:
from openfermion.utils import SpinPairs
tunneling = [('neighbor', (0, 0), 1.)] # Not sure if this is right
interaction = [('onsite', (0, 0), 2., SpinPairs.DIFF)] # Not sure if this is right
potential = None 
mag_field = 0. 
particle_hole_sym = False # Not sure if this is right

In [6]:
hubbard = FermiHubbardModel(lattice , tunneling_parameters=tunneling, interaction_parameters=interaction, 
                            potential_parameters=potential, magnetic_field=mag_field, 
                            particle_hole_symmetry=particle_hole_sym)

In [7]:
# Test: Make sure the above Hamiltonian is iso-spectral with a Hubbard Hamiltonian with same parameters 
from openfermion.hamiltonians import fermi_hubbard
from openfermion.utils import eigenspectrum
test_hub = fermi_hubbard(x_n, y_n, 1. , 2., chemical_potential=0.0, magnetic_field=mag_field, 
                         periodic=periodic, spinless=spinless, particle_hole_symmetry=particle_hole_sym)
print("Spectrums are the same: ", array_eq(eigenspectrum(test_hub), eigenspectrum(hubbard.hamiltonian()), tol))

Spectrums are the same:  True


In [ ]:
"""
The full FermionOperator of the Hubbard model is in hubbard.ham. 
We also need to break the non-interacting part into 4 terms: horizontal even hopping, horizontal odd opping, 
vertical even hopping, and vertical odd hopping. 
I do this with the function _gen_non_interacting_dict(ham), which returns a dictionary with 4 keys 
corresponding to the values we want. 
"""
hubbard = {
    ham: fermi_hubbard(x_n, y_n, tunneling, coulomb, chem_potential, 
                        mag_field, periodic, spinless), 
    ham_non_interacting: fermi_hubbard(x_n, y_n, tunneling, 0, chem_potential, 
                                        mag_field, periodic, spinless), 
    ham_non_interacting_dict: _gen_non_interacting_dict(hubbard.ham_non_interacting), 
    hor_even: ham_non_interacting_dict['hor_even'], 
    hor_odd: ham_non_interacting_dict['hor_odd'], 
    ver_even: ham_non_interacting_dict['ver_even'], 
    ver_odd: ham_non_interacting_dict['ver_odd']
}

def _gen_non_interacting_dict(ham):
    """
    Return a dictonary of FermionOperators with keys hor_even, hor_odd, ver_even, ver_odd. 
    ham: FermionOperator 
    """
    for term_tuple in ham.terms: 
        

In [ ]:
eig = np.linalg.eigh(x.hamiltonian)
index = np.argmin(eig[0])
print('Ground state energy: ', eig[0][index])
gstate = np.transpose(NKron(eig[1][:,index]))
gstate = gstate / np.linalg.norm(gstate)
print('Ground state: ', gstate)
print(gstate.shape)

In [ ]:
np.dot(np.transpose(np.conj(gstate)), np.dot(x.hamiltonian, gstate))

## Variational Hamiltonian Ansatz

The VHA is an ansatz deeply connected to time-evolution of the system. It splits the Hamiltonian into sub-operators and then does time-evolution for those operators: 
$$\large U(\theta) = \prod_{k=1}^n \prod_{\alpha=1}^N \exp \Big( i\theta_{\alpha, k} H_{\alpha} \Big) $$
where $H_\alpha$ are the sub-Hamiltonians and $\theta$ are the parameters being optimized. 

**Question:** Is product from $k=1$ to $n$ meant to indicate multiple iterations of the algorithm? So are there only $N$ parameters, or are there $n \cdot N$? I think it's the former, because $\Big( \exp (i \theta_{1 k} H_1) \cdots \exp (i \theta_{N k} H_N) \Big)$ commutes with itself, so we can combine terms.  

**Question:** Why is this a good ansatz? It seems it only has access to states that are evolutions of the state we start with. Why is that a guarantee that it'll approximate the ground state? 

In [1811.04476](https://arxiv.org/pdf/1811.04476.pdf) they use $N=5$, splitting it as I did above: even and odd horizontal hopping terms, even and odd vertical horizontal terms, and the on-site interaction terms. 

In [ ]:
def time_evo(ham, t): 
    # returns e^{i*t*ham}
    # scipy.linalg.expm() uses the Pade approximant; try to understand more of how it works because it seems 
    # pretty useful: https://en.wikipedia.org/wiki/Pad%C3%A9_approximant
    return scipy.linalg.expm(-1j * t * ham)

def prod_time_evo(ham_sets, ts):
    result = 1 
    for pair in zip(ham_sets, ts): 
        result *= time_evo(pair[0], pair[1])
    return result

def test_linalg_expm():
    return scipy.linalg.expm(X)

test_linalg_expm()

In [ ]:
scipy.linalg?

In [ ]:
y = [1, 1]
2 * y

In [ ]:
class VHA: 
    def __init__(self, hubbard, start_state): 
        self.ham_sets = hubbard.ham_sets 
        self.hamiltonian = hubbard.hamiltonian
        self.start_state = start_state
        self.start_params = np.array([5e30 for i in range(5)])
            
    def optimize(self):  
        self.energies = []
        
        def energy(t_params): 
            unitary = prod_time_evo(2 * self.ham_sets, np.real(t_params))
            state = np.dot(unitary, self.start_state)
            curr_energy = np.dot(np.transpose(np.conj(state)), np.dot(self.hamiltonian, state))[0][0]
            #print(curr_energy)
            self.energies.append(curr_energy)
            return curr_energy
                    
        self.sol = scipy.optimize.minimize(energy, 2 * self.start_params, method='Powell', 
                                           bounds=[(0, 3) for i in range(2 * len(self.start_params))])

In [ ]:
vha = VHA(x, gstate)

In [ ]:
vha.optimize()

In [ ]:
print('Minimum energy: ', vha.sol.fun)
print('Number of iterations: ', vha.sol.nfev)
est_g_state = np.dot(prod_time_evo(vha.ham_sets, vha.sol.x), vha.start_state)
print('Estimated ground state: ', est_g_state)
print(vha.sol.x)

### Finding Ground State of Non-interacting Terms

VHA works because finding the ground state of non-interacting terms is efficiently computable on a QC and the VHA can evolve from that ground state to the ground state of the entire Hubbard Hamiltonian. 

We'll start by finding the ground state by using numpy to see if this actually work. 

In [ ]:
non_interacting_ham = 0
for i in range(4): 
    non_interacting_ham += x.ham_sets[i]

In [ ]:
non_int_ham_noise = non_interacting_ham + np.random.normal(size=(256,256))
eig_non = np.linalg.eig(non_int_ham_noise)
index = np.argmin(eig_non[0])
genergy_non = eig_non[0][index]
print('index', index)
print('Lowest energy: ', genergy_non)
gstate_non = np.transpose(NKron(eig_non[1][:,index]))
print(gstate_non.shape)
print(non_int_ham_noise.shape)

In [ ]:
eig_non[0][index]

In [ ]:
test_prod = np.dot(non_int_ham_noise, gstate_non)
conj = np.conjugate(gstate_non)
res = 0
for i in range(len(conj)):
    con = np.real(conj[i])
    pro = np.real(test_prod[i])
    if np.abs(con) < 1e-20: 
        continue
    elif np.abs(pro) < 1e-20: 
        continue
    else: 
        res += con * pro
#     print(con)
#     print(pro)
#     print(res)
#     print('---') 
#print('Where does the e-31 come from?!?')
res

In [ ]:
# Choose one of the eigenvectors from lowest eigenvalue
vha = VHA(x, gstate_non)
vha.optimize()

In [ ]:
vha.sol.fun

In [ ]:
est_g_state = np.dot(prod_time_evo(2 * vha.ham_sets, vha.sol.x), vha.start_state)
overlap = np.dot(adjoint(est_g_state), gstate)
overlap[0][0]

In [ ]:
vha.sol.x

In [ ]:
for ind in range(0, 80): 
    vha = VHA(x, np.transpose(NKron(eig_non[1][:,ind])))
    vha.optimize()
    print(ind, ' : ', np.real(vha.sol.fun))
    if np.real(vha.sol.fun) < -5: 
        print('Got it! Index is', ind)